# Aprendizaje Automático
## Tercera práctica guiada
---
Para la tercera práctica del curso de aprendizaje automático, se utilizará el mismo conjunto de datos que pertenece al repositorio especial de Machine Learning de la [UCI](https://archive.ics.uci.edu/ml). El conjunto de datos se llama "Adult" y toda la información correspondiente al mismo se encuentra [aquí](https://archive.ics.uci.edu/ml/datasets/Adult).

---
### Máquinas de Vectores de Soporte, K-Vecinos más cercanos y Análisis de discriminante lineal y cuadrático.

###### En principio, importamos todas las librerías necesarias para trabajar Aprendizaje Automático con Python y Sci-kit learn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
import itertools
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import tree
from sklearn import svm
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
import io
import requests


###### Luego procedemos a importar los datos de entrenamiento y eliminar los valores faltantes, además de convertir los valores categóricos a enteros.

In [2]:
url="https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
ds_ = pd.read_csv(url, header=None, na_values=" ?")
ds = ds_.dropna()
ds = ds.rename(columns = {ds.columns.values[14]:'C'}) # Cambiamos el nombre de la última columna por "C" de clase
le = preprocessing.LabelEncoder() # Label encoder de sci-kit
ds = ds.apply(le.fit_transform) # Convertimos los valores de object a int
class_names = ['Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-num', 'Marital-status', 'Occupation',
              'Relationship', 'Race', 'Sex', 'Capital-gain', 'Capital-loss', 'Hpw', 'Country']
ds


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,C
0,22,5,2491,9,12,4,0,1,4,1,24,0,39,38,0
1,33,4,2727,9,12,2,3,0,4,1,0,0,12,38,0
2,21,2,13188,11,8,0,5,1,4,1,0,0,39,38,0
3,36,2,14354,1,6,2,5,0,2,1,0,0,39,38,0
4,11,2,18120,9,12,2,9,5,2,0,0,0,39,4,0
5,20,2,16567,12,13,2,3,5,4,0,0,0,39,38,0
6,32,2,7982,6,4,3,7,1,2,0,0,0,15,22,0
7,35,4,12746,11,8,2,3,0,4,1,0,0,44,38,1
8,14,2,1225,12,13,4,9,1,4,0,104,0,49,38,1
9,25,2,7908,9,12,2,3,0,4,1,78,0,39,38,1


###### Importamos los datos de prueba de la misma manera que importamos los datos de entrenamiento.

In [4]:
urltest = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
test_ = pd.read_csv(urltest, skiprows=1, header=None, na_values=" ?")
test = test_.dropna()
test = test.rename(columns = {test.columns.values[14]:'C'}) # Cambiamos el nombre de la última columna por "C" de clases = test.apply(le.fit_transform) # Convertimos los valores de object a int
test = test.apply(le.fit_transform) # Convertimos los valores de object a int
test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,C
0,8,2,8315,1,6,4,6,3,2,1,0,0,39,37,0
1,21,2,1754,11,8,2,4,0,4,1,0,0,49,37,0
2,11,1,10750,7,11,2,10,0,4,1,0,0,39,37,1
3,27,2,4780,15,9,2,6,0,2,1,87,0,39,37,1
5,17,2,7091,0,5,4,7,1,4,1,0,0,29,37,0
7,46,4,2342,14,14,2,9,0,4,1,44,0,31,37,1
8,7,2,11153,15,9,4,7,4,4,0,0,0,39,37,0
9,38,2,2361,5,3,2,2,0,4,1,0,0,9,37,0
10,48,2,6199,11,8,2,6,0,4,1,77,0,39,37,1
11,19,0,7773,9,12,2,0,0,4,1,0,0,39,37,0


###### En este taller se debe extraer aleatoriamente un 10% de los datos del conjunto de entrenamiento. Este subconjunto constituirá el conjunto de validación, que se usará para estimar los valores de los parámetros en las técnicas que así lo requieran. Todas las técnicas deberán entrenarse con el mismo conjunto de entrenamiento para que sea posible comparar sus desempeños. En la siguiente celda, se divide el conjunto de entrenamiento como se enunció.

In [14]:
x_train = ds.drop('C', axis = 1)
y_train = ds['C']

x_test = test.drop('C', axis = 1)
y_test = test['C']

from collections import Counter

x_strainval, x_stestval, y_strainval, y_stestval = train_test_split(x_train, y_train, test_size = 0.1, random_state=None)
print('Original dataset shape trainval {}'.format(Counter(y_strainval)))
print('Original dataset shape testval {}'.format(Counter(y_stestval)))
x_train

Original dataset shape trainval Counter({0: 20370, 1: 6775})
Original dataset shape testval Counter({0: 2284, 1: 733})


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,22,5,2491,9,12,4,0,1,4,1,24,0,39,38
1,33,4,2727,9,12,2,3,0,4,1,0,0,12,38
2,21,2,13188,11,8,0,5,1,4,1,0,0,39,38
3,36,2,14354,1,6,2,5,0,2,1,0,0,39,38
4,11,2,18120,9,12,2,9,5,2,0,0,0,39,4
5,20,2,16567,12,13,2,3,5,4,0,0,0,39,38
6,32,2,7982,6,4,3,7,1,2,0,0,0,15,22
7,35,4,12746,11,8,2,3,0,4,1,0,0,44,38
8,14,2,1225,12,13,4,9,1,4,0,104,0,49,38
9,25,2,7908,9,12,2,3,0,4,1,78,0,39,38


###### Teniendo el subconjunto de validación, se entrenarán varias SVM con el fin de poder estimar cuál es la que mejor rendimiento muestra dependiendo de los distintos parámetros que se pueden ajustar para cada una.

A continuación se estimará qué parámetro C es el que mejor se comporta con las SVM.

In [11]:
########################################################################
#################### Linear SVM C=1.0 ##################################

SVM1 = svm.LinearSVC(dual=False)
SVM1.fit(x_strainval, y_strainval)

SVM1_predict = SVM1.predict(x_stestval)

print("\n############## Accuracy Linear SVM C=1.0 ##################")
print(metrics.accuracy_score(y_stestval, SVM1_predict))
print(metrics.confusion_matrix(y_stestval, SVM1_predict))

########################################################################
#################### Linear SVM C=5.0 ##################################

SVM5 = svm.LinearSVC(dual=False, C=5.0)
SVM5.fit(x_strainval, y_strainval)

SVM5_predict = SVM5.predict(x_stestval)

print("\n############## Accuracy Linear SVM C=5.0 ##################")
print(metrics.accuracy_score(y_stestval, SVM5_predict))
print(metrics.confusion_matrix(y_stestval, SVM5_predict))

########################################################################
#################### Linear SVM C=10.0 ##################################

SVM10 = svm.LinearSVC(dual=False, C=10.0)
SVM10.fit(x_strainval, y_strainval)

SVM10_predict = SVM10.predict(x_stestval)

print("\n############## Accuracy Linear SVM C=10.0 ##################")
print(metrics.accuracy_score(y_stestval, SVM10_predict))
print(metrics.confusion_matrix(y_stestval, SVM10_predict))

########################################################################
#################### Linear SVM C=50.0 ##################################

SVM50 = svm.LinearSVC(dual=False, C=50.0)
SVM50.fit(x_strainval, y_strainval)

SVM50_predict = SVM50.predict(x_stestval)

print("\n############## Accuracy Linear SVM C=50.0 ##################")
print(metrics.accuracy_score(y_stestval, SVM50_predict))
print(metrics.confusion_matrix(y_stestval, SVM50_predict))

########################################################################
#################### Linear SVM C=50.0 ##################################

SVM50 = svm.LinearSVC(dual=False, C=500.0)
SVM50.fit(x_strainval, y_strainval)

SVM50_predict = SVM50.predict(x_stestval)

print("\n############## Accuracy Linear SVM C=50.0 ##################")
print(metrics.accuracy_score(y_stestval, SVM50_predict))
print(metrics.confusion_matrix(y_stestval, SVM50_predict))
x_stestval


############## Accuracy Linear SVM C=1.0 ##################
0.799469671859463
[[2154  107]
 [ 498  258]]

############## Accuracy Linear SVM C=5.0 ##################
0.799469671859463
[[2154  107]
 [ 498  258]]

############## Accuracy Linear SVM C=10.0 ##################
0.799469671859463
[[2154  107]
 [ 498  258]]

############## Accuracy Linear SVM C=50.0 ##################
0.799469671859463
[[2154  107]
 [ 498  258]]

############## Accuracy Linear SVM C=50.0 ##################
0.799469671859463
[[2154  107]
 [ 498  258]]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
22434,28,2,8662,12,13,0,9,1,4,1,0,0,39,38
4211,4,2,14871,11,8,2,4,0,4,1,0,0,39,38
13615,0,2,11248,1,6,4,11,4,4,0,0,0,11,30
6261,24,2,10595,11,8,5,0,4,4,0,0,0,39,38
13202,20,2,4206,9,12,2,9,0,4,1,0,0,54,38
14082,9,1,14790,11,8,2,10,0,4,1,49,0,39,38
27397,15,1,4762,9,12,0,9,4,4,0,0,0,34,38
1981,32,0,8661,11,8,2,0,0,4,1,0,0,39,38
3848,11,2,16473,15,9,5,12,4,4,1,0,0,39,38
27413,38,2,4826,14,14,2,3,0,4,1,117,0,39,38


###### Claramente al variar el parámetro C no se encontró cambio alguno en el rendimiento de la SVM Lineal. Ahora, se entrenarán 2 SVM con funciones kernel distintas para evaluar cuál parametro arroja los mejores resultados con la partición previa que se obtuvo.

In [6]:
########################################################################
########################### RBF SVM ####################################

SVMrbf = svm.SVC(kernel='rbf')
SVMrbf.fit(x_strainval, y_strainval)

SVMrbf_predict = SVMrbf.predict(x_stestval)

print("\n############## Accuracy SVM RBF ##################")
print(metrics.accuracy_score(y_stestval, SVMrbf_predict))
print(metrics.confusion_matrix(y_stestval, SVMrbf_predict))

########################################################################
########################## Sigmoid SVM #################################

SVMsigmoid = svm.SVC(kernel='sigmoid')
SVMsigmoid.fit(x_strainval, y_strainval)

SVMsigmoid_predict = SVMsigmoid.predict(x_stestval)

print("\n############## Accuracy SVM Sigmoid ##################")
print(metrics.accuracy_score(y_stestval, SVMsigmoid_predict))
print(metrics.confusion_matrix(y_stestval, SVMsigmoid_predict))


############## Accuracy SVM RBF ##################
0.7494199535962877
[[2257    2]
 [ 754    4]]

############## Accuracy SVM Sigmoid ##################
0.7487570434206166
[[2259    0]
 [ 758    0]]


###### La SVM con kernel lineal obtuvo un mejor accuracy y por tal razón será usada para entrenar el clasificador y posteriormente evaluarlo con los datos originales de prueba. En la siguiente porción de código, entrenamos el clasificador y realizamos la respectiva predicción. Además, evaluamos el desempeño del clasificador teniendo en cuenta las métricas que provee el reporte de clasificación de sklearn.

In [13]:
########################################################################
#################### Linear SVM FINAL ##################################

SVM = svm.LinearSVC(dual=False)
SVM.fit(x_train, y_train)

SVM_predict = SVM.predict(x_test)

print("\n############## Accuracy Linear SVM FINAL ##################")
print(metrics.accuracy_score(y_test, SVM_predict))
print(metrics.confusion_matrix(y_test, SVM_predict))
x_test


############## Accuracy Linear SVM FINAL ##################
0.80199203187251
[[10697   663]
 [ 2319  1381]]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,8,2,8315,1,6,4,6,3,2,1,0,0,39,37
1,21,2,1754,11,8,2,4,0,4,1,0,0,49,37
2,11,1,10750,7,11,2,10,0,4,1,0,0,39,37
3,27,2,4780,15,9,2,6,0,2,1,87,0,39,37
5,17,2,7091,0,5,4,7,1,4,1,0,0,29,37
7,46,4,2342,14,14,2,9,0,4,1,44,0,31,37
8,7,2,11153,15,9,4,7,4,4,0,0,0,39,37
9,38,2,2361,5,3,2,2,0,4,1,0,0,9,37
10,48,2,6199,11,8,2,6,0,4,1,77,0,39,37
11,19,0,7773,9,12,2,0,0,4,1,0,0,39,37


###### Ahora, se crea el clasificador para la técnica de K-Vecinos más cercanos y se evalúa el desempeño del clasificador.

In [47]:
################################################################
##################  K-Vecinos más cercanos #####################
################################################################

n_neighbors = 21
KNN = KNeighborsClassifier(n_neighbors)
KNN.fit(x_train, y_train)
KNN_predict = KNN.predict(x_test)

print("\n############## Accuracy KNN ##################")
print(metrics.accuracy_score(y_test, KNN_predict))
print(metrics.confusion_matrix(y_test, KNN_predict))


############## Accuracy KNN ##################
0.7903718459495352
[[11141   219]
 [ 2938   762]]


###### Ahora, se crea el clasificador para las técnicas de Análisis de discriminante lineal y se evalúa el desempeño del clasificador.

In [9]:
################################################################
#############  Análisis de discriminante lineal ################
################################################################

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)

y_predl = lda.predict(x_test)
print("\n############## Accuracy LDA ##################")
print(metrics.accuracy_score(y_test, y_predl))
print(metrics.confusion_matrix(y_test, y_predl))



################################################################
############  Análisis de discriminante cuadrático #############
################################################################

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_train, y_train)

y_predd = qda.predict(x_test)
print("\n############## Accuracy QDA ##################")
print(metrics.accuracy_score(y_test, y_predd))
print(metrics.confusion_matrix(y_test, y_predd))


############## Accuracy LDA ##################
0.8177954847277557
[[10798   562]
 [ 2182  1518]]

############## Accuracy QDA ##################
0.8200531208499336
[[10549   811]
 [ 1899  1801]]
